In [ ]:
import tensorflow as tf
from RL_brain import DeepQNetwork
import numpy as np
from env import Env
import matplotlib.pyplot as plt
from Configuration import printPeriodic,setDbgPrint,null


In [ ]:
sim = {
    'cfg' : {
        'run' : { # run parameters
            'numEpisodes'    : 1, # 1,        # num. of episodes to run
            'maxSteps'       : 10000,         # max. num. of sim. steps
            'startLearnStep' : 200,           # start periodic training after this step num.
            'learnPeriod'    : 5,             # num. of steps between (re)training
            'dbgPrint'       : printPeriodic, # function or string name of function for debug print output
            'statusPeriod'   : 100,           # output program status at this interval
        },
        'dqn': {  # DQN parameters
            'learningRate': 0.01,  # please provide summary comments for each parameter
            'rewardDecay': 0.9,
            'eGreedy': 1,
            'eGreedyincrement': 0.005,  # Change the e_greedy value
            'replaceTargetIter': 200,
            'memorySize': 2000,
        }
    }
}

In [ ]:
class run():
    def __init__(
            self,
            n_actions,
            n_features,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.built_net()
        self.sess = tf.Session()
        self.saver = tf.train.Saver()
        self.saver.restore(self.sess, 'my_net/my_test_model.ckpt') #Load parameter
        self.params = []
        self.cost_his = []
    def built_net(self):
        # Reconstruction neural network model.Only one of the neural networks is kept here.
        c_names, n_l1, w_initializer, b_initializer = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES], 10, \
                                                      tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)
        self.env_state_ = tf.placeholder(tf.float32, [None, self.n_features], name='env_state_')
        with tf.variable_scope('target_net'):
            c_names = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES]

            with tf.variable_scope(
                    'l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1], initializer=w_initializer, collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1], initializer=b_initializer, collections=c_names)
                l1 = tf.nn.tanh(tf.matmul(self.env_state_, w1) + b1)
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions], initializer=w_initializer, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions], initializer=b_initializer, collections=c_names)
                self.q_next = tf.matmul(l1, w2) + b2

    def choose_action(self, env_state):
        env_state = env_state[np.newaxis, :]
        actions_value = self.sess.run(self.q_next,feed_dict={self.env_state_: env_state})
        action = np.argmax(actions_value)
        return action

    def run_(self,cfg=None):
        numEpisodes = cfg['numEpisodes']
        maxSteps = cfg['maxSteps']
        dbgPrint = cfg.get('dbgPrint', null)

        statusPeriod = cfg.get('statusPeriod', 1)
        for episode in range(1,numEpisodes+1):
            step = 0
            env_state_1 = env.reset()
            while step < maxSteps:
                env_state = np.hstack((env_state_1["Channel_1"], env_state_1["Channel_6"],
                                       env_state_1["Channel_11"]))

                action = self.choose_action(env_state)
                if action == 0:
                    action_ = "Channel_1"
                elif action == 1:
                    action_ = "Channel_6"
                else:
                    action_ = "Channel_11"
                observation = env_state_1[action_]
                dbgPrint('{}: action_ = {}, observation = {}\n'.format(env.time, action_, observation),
                         period=statusPeriod, counter=env.time)
                print('{}: action_ = {}, observation = {}\n'.format(env.time, action_, observation))
                env_state_, observation_, reward = env.step(action_)
                env_state_ = np.hstack(( env_state_["Channel_1"],  env_state_["Channel_6"],
                                env_state_["Channel_11"]))
                env_state_ = {"Channel_1": env_state_[0:4],
                              "Channel_6": env_state_[4:8],
                              "Channel_11": env_state_[8:12]

                              }
                env_state_1 = env_state_
                step += 1
